# ADM Health Check

The ADM Health Check serves as your first line of diagnostics for Adaptive Decision Manager (ADM) implementations. It provides a comprehensive initial assessment that helps identify potential issues, bottlenecks, and areas requiring attention in your ADM system. Think of it as an initial diagnostic scan that highlights where you might need to look deeper.

## What Can Health Check Help You Find?

The Health Check report helps identify various potential issues in your ADM system:

1. **Underperforming Models**
   - Models with low AUC
   - Models with insufficient responses(learning data)
   - Unexpected performance degradation over time

2. **Data Quality Issues**
   - Missing responses in certain channels
   - Unexpected gaps in data collection
   - Irregular response patterns

3. **Predictor Problems**
   - Inactive or stale predictors
   - Predictors with no responses
   - Poor performing predictors

4. **Action/Channel Concerns**
   - Underutilized channels
   - Actions with low success rates
   - Imbalanced response distribution

## Example: Identifying Problematic Models

Let's look at how the Health Check can help identify models that need attention. We'll use the response distribution analysis as an example:

In [1]:
import polars as pl
from pdstools import datasets

# Load sample data
dm = datasets.cdh_sample()

df_channel_overview = (
    dm.aggregates.summary_by_channel()
    .with_columns(
        NBAD=pl.when(pl.col("usesNBAD"))
        .then(
            pl.when(pl.col("usesNBADOnly"))
            .then(pl.lit("Yes"))
            .otherwise(pl.lit("With additional configurations"))
        )
        .otherwise(pl.lit("No"))
    )
    .drop(
        [
            "ChannelDirectionGroup",
            "ChannelDirection",
            "DateRange Min",
            "DateRange Max",
        ]
    )
    .collect()
)

In [2]:
from great_tables import style, html, loc
from pdstools.utils import report_utils
from pdstools.adm.CDH_Guidelines import CDHGuidelines

cdh_guidelines = CDHGuidelines()

formatted_channel_overview = (
    report_utils.table_standard_formatting(
        df_channel_overview,
        cdh_guidelines=cdh_guidelines,
        highlight_limits={
            "Positives": "Positive Responses",
            "Performance": "Model Performance",
            "ResponseCount": "Responses",
            "Total Number of Actions": "Actions",
            "Used Actions": "Actions",
            "Total Number of Treatments": "Treatments",
            "Used Treatments": "Treatments",
            "Issues": "Issues",
        },
        highlight_lists={
            "Channel": cdh_guidelines.standard_channels,
            "Direction": cdh_guidelines.standard_directions,
        },
        highlight_configurations=["Configuration"],
    )
    .fmt_percent(decimals=0, columns=["OmniChannel Actions"])
    .fmt_number(decimals=2, columns=["Performance"])
    .fmt_percent(decimals=2, columns=["CTR"])
    .cols_label(
        CTR="Base Rate",
        ResponseCount="Total Responses",
        Positives="Total Positives",
        Configuration="Supported by Configurations",
        Performance="Average Performance",
    )
    .tab_spanner(
        label=html("<b>ADM Models</b>"),
        columns=["Positives", "ResponseCount", "Performance", "Configuration"],
    )
    .tab_spanner(
        label=html("<b>NBAD Setup</b>"),
        columns=[
            "Total Number of Actions",
            "Total Number of Treatments",
            "Used Actions",
            "Used Treatments",
            "Issues",
            "Groups",
            "NBAD",
            "OmniChannel Actions",
        ],
    )
)

In [4]:
display(
    formatted_channel_overview.cols_hide(
        [
            "Positives",
            "ResponseCount",
            "Performance",
            "Configuration",
            "CTR",
            "isValid",
            "usesNBAD",
            "usesNBADOnly",
        ]
    ).tab_style(
        style=style.text(decorate="line-through"),
        locations=loc.body(rows=pl.col("isValid").not_()),
    )
)

GT(_tbl_data=shape: (3, 18)
┌─────────┬───────────┬───────────┬──────────────┬───┬─────────────┬─────────────┬──────────┬──────┐
│ Channel ┆ Direction ┆ Positives ┆ ResponseCoun ┆ … ┆ usesNBADOnl ┆ OmniChannel ┆ CTR      ┆ NBAD │
│ ---     ┆ ---       ┆ ---       ┆ t            ┆   ┆ y           ┆ Actions     ┆ ---      ┆ ---  │
│ cat     ┆ cat       ┆ f32       ┆ ---          ┆   ┆ ---         ┆ ---         ┆ f32      ┆ str  │
│         ┆           ┆           ┆ f32          ┆   ┆ bool        ┆ f64         ┆          ┆      │
╞═════════╪═══════════╪═══════════╪══════════════╪═══╪═════════════╪═════════════╪══════════╪══════╡
│ Email   ┆ Outbound  ┆ 9497.0    ┆ 462235.0     ┆ … ┆ true        ┆ 0.604167    ┆ 0.020546 ┆ Yes  │
│ SMS     ┆ Outbound  ┆ 51558.0   ┆ 659196.0     ┆ … ┆ true        ┆ 0.592593    ┆ 0.078213 ┆ Yes  │
│ Web     ┆ Inbound   ┆ 45665.0   ┆ 438818.0     ┆ … ┆ true        ┆ 0.763158    ┆ 0.104064 ┆ Yes  │
└─────────┴───────────┴───────────┴──────────────┴───┴─────────────┴─────────────┴──────────┴──────┘, _body=<great_tables._gt_data.Body object at 0x30ef3b2e0>, _boxhead=Boxhead([ColInfo(var='Channel', type=<ColInfoTypeEnum.default: 1>, column_label='Channel', column_align='center', column_width=None), ColInfo(var='Direction', type=<ColInfoTypeEnum.default: 1>, column_label='Direction', column_align='center', column_width=None), ColInfo(var='Positives', type=<ColInfoTypeEnum.hidden: 4>, column_label='Total Positives', column_align='right', column_width=None), ColInfo(var='ResponseCount', type=<ColInfoTypeEnum.hidden: 4>, column_label='Total Responses', column_align='right', column_width=None), ColInfo(var='Performance', type=<ColInfoTypeEnum.hidden: 4>, column_label='Average Performance', column_align='right', column_width=None), ColInfo(var='Configuration', type=<ColInfoTypeEnum.hidden: 4>, column_label='Supported by Configurations', column_align='left', column_width=None), ColInfo(var='Total Number of Actions', type=<ColInfoTypeEnum.default: 1>, column_label='Total Number of Actions', column_align='center', column_width=None), ColInfo(var='Total Number of Treatments', type=<ColInfoTypeEnum.default: 1>, column_label='Total Number of Treatments', column_align='center', column_width=None), ColInfo(var='Used Actions', type=<ColInfoTypeEnum.default: 1>, column_label='Used Actions', column_align='center', column_width=None), ColInfo(var='Used Treatments', type=<ColInfoTypeEnum.default: 1>, column_label='Used Treatments', column_align='center', column_width=None), ColInfo(var='Issues', type=<ColInfoTypeEnum.default: 1>, column_label='Issues', column_align='center', column_width=None), ColInfo(var='Groups', type=<ColInfoTypeEnum.default: 1>, column_label='Groups', column_align='center', column_width=None), ColInfo(var='NBAD', type=<ColInfoTypeEnum.default: 1>, column_label='NBAD', column_align='left', column_width=None), ColInfo(var='OmniChannel Actions', type=<ColInfoTypeEnum.default: 1>, column_label='OmniChannel Actions', column_align='right', column_width=None), ColInfo(var='isValid', type=<ColInfoTypeEnum.hidden: 4>, column_label='isValid', column_align='center', column_width=None), ColInfo(var='usesNBAD', type=<ColInfoTypeEnum.hidden: 4>, column_label='usesNBAD', column_align='center', column_width=None), ColInfo(var='usesNBADOnly', type=<ColInfoTypeEnum.hidden: 4>, column_label='usesNBADOnly', column_align='center', column_width=None), ColInfo(var='CTR', type=<ColInfoTypeEnum.hidden: 4>, column_label='Base Rate', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x30ef3a6e0>, _spanners=Spanners([SpannerInfo(spanner_id='<b>ADM Models</b>', spanner_level=0, spanner_label=Html(text='<b>ADM Models</b>'), spanner_units=None, spanner_pattern=None, vars=['Positives', 'ResponseCount', 'Performance', 'Configuration'], built=None), SpannerInfo(spanner_id='<b>NBAD Setup</b>', spanner_level=0, spanner_label=Html(text='<b>NBAD Setup</b>'), spanner_units=None, spanner_pattern=None, va

In this example, we're looking for models that might need attention due to:
- Insufficient response volume (< 1000 responses)
- Poor performance (AUC < 0.55)

The response distribution visualization helps identify:
- Models with unusual response patterns
- Potential data collection issues
- Seasonal or temporal anomalies

## Running a Health Check

You can generate an ADM Health Check report in two ways:

1. Using the Python function:
```python
from pdstools.generate import healthcheck

healthcheck(
    input_dir="path/to/your/datamart/exports",
    output_dir="path/to/output",
    title="My ADM Health Check"
)
```

2. Through the pdstools app via CLI:
```bash
uv run pdstools run
```

## Conclusion

The ADM Health Check is your first step in maintaining a healthy ADM implementation. Use it regularly to:
- Catch potential issues early
- Identify areas needing deeper investigation
- Maintain optimal system performance
- Guide your optimization efforts

When issues are found, conduct more specific analysis by either in pdstools or in with your favourite analysis tool to investigate further and determine the best course of action.